In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
Raw['tf'].unique()

In [ ]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['A']        = Raw['a']
    Src['Z']        = Raw['z']
    Src['Symbol']   = Raw['symbol']
    Src['TF']       = Raw['tf']
    
    Src['Datetime'] = Raw['datetime']
    Src['Date']     = Raw['datetime'].dt.date
    Src['Time']     = Raw['datetime'].dt.time
    
    Src['Ticks']    = Raw['tick_volume']
    Src['Volume']   = Raw['real_volume']
    Src['Price']    = Raw['close']
    
    Src['Open']     = Raw['open']
    Src['High']     = Raw['high']
    Src['Low']      = Raw['low']
    Src['Close']    = Raw['close']

    Src['Direct']       = (Src['Close'] - Src['Open']).apply(lambda x: -1 if (x <  0) else  +1)
    Src['Bull Sign']    =                Src['Direct'].apply(lambda x:  x if (x >= 0) else nan)
    Src['Bear Sign']    =                Src['Direct'].apply(lambda x:  x if (x <  0) else nan)

    Src['Change']       = Src['Close'] - Src['Open']
    Src['HL']           = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Raw)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Psc']         = Calc.groupby(['Symbol','TF'], sort=0)['Change'] .expanding(20).rank(ascending=1, pct=1).mul(100)      .reset_index(drop=1)
    Calc['HL Psc']          = Calc.groupby(['Symbol','TF'], sort=0)['HL']     .expanding(20).rank(ascending=1, pct=1).mul(100)      .reset_index(drop=1)
    
    Calc['Chg Norm']        = Calc.apply(lambda x: x['Change'] if (5 < x['Chg Psc'] < 95) else nan, axis=1)
    Calc['HL Norm']         = Calc.apply(lambda x: x['HL']     if (5 < x['HL Psc']  < 95) else nan, axis=1)

    Calc['Chg Norm Abs']    = Calc['Chg Norm'].abs()
    Calc['Chg Norm Pos']    = Calc['Chg Norm'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Chg Norm Neg']    = Calc['Chg Norm'].apply(lambda x: x if (x <  0) else nan)
    
    Calc['HL Norm Pos']     = Calc['HL Norm'] * Calc['Bull Sign']
    Calc['HL Norm Neg']     = Calc['HL Norm'] * Calc['Bear Sign']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [15]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm']].describe().round(0).astype(int)

Chg Norm                                        
              count mean   std   min   25%  50%   75%   max
Symbol TF                                                  
WIN$   M5     93314    0    99  -366   -61    0    62   366
       M15    43035   -1   193  -556  -118    0   118   564
       H1     11089   -5   395 -1237  -242    0   242  1044
       H3      4117  -14   666 -2114  -374    5   362  1760
       D1      1142  -42  1387 -4056 -1011   29   955  2983
       W1       224  -45  2828 -6542 -1994 -155  2148  6187

In [16]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Abs']].describe().round(0).astype(int)

Chg Norm Abs                                       
                  count  mean   std min  25%   50%   75%   max
Symbol TF                                                     
WIN$   M5         93314    78    62   0   28    62   113   366
       M15        43035   150   121   0   53   118   219   564
       H1         11089   306   250   0  106   242   445  1237
       H3          4117   505   435   0  150   370   767  2114
       D1          1142  1129   806   0  464   972  1621  4056
       W1           224  2309  1625   0  970  2096  3336  6542

In [17]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Pos']].describe().round(0).astype(int)

Chg Norm Pos                                        
                  count  mean   std min   25%   50%   75%   max
Symbol TF                                                      
WIN$   M5         48186    75    62   0    26    60   110   366
       M15        21868   146   120   0    50   115   216   564
       H1          5607   298   242   0   101   239   432  1044
       H3          2094   482   413   0   140   355   740  1760
       D1           584  1062   757   0   436   942  1591  2983
       W1           105  2415  1577   0  1113  2354  3326  6187

In [18]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Neg']].describe().round(0).astype(int)

Chg Norm Neg                                       
                  count  mean   std   min   25%   50%  75% max
Symbol TF                                                     
WIN$   M5         45128   -80    62  -366  -115   -64  -31  -5
       M15        21167  -153   122  -556  -222  -120  -56  -5
       H1          5482  -315   257 -1237  -458  -247 -111  -5
       H3          2023  -528   456 -2114  -789  -387 -155  -5
       D1           558 -1198   850 -4056 -1652 -1046 -508  -6
       W1           119 -2216  1667 -6542 -3325 -1913 -826 -63

In [19]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm']].describe().round(0).astype(int)

HL Norm                                           
             count  mean   std   min   25%   50%   75%    max
Symbol TF                                                    
WIN$   M5    86773   201    94    70   129   178   253    654
       M15   39491   396   183   113   253   352   500   1046
       H1    10054   791   362   245   509   708  1004   1994
       H3     3761  1283   640   235   765  1163  1684   3338
       D1     1021  2603   900  1163  1909  2438  3146   5646
       W1      215  5677  1868  2831  4156  5197  7068  12323

In [20]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm Pos']].describe().round(0).astype(int)

HL Norm Pos                                           
                 count  mean   std   min   25%   50%   75%    max
Symbol TF                                                        
WIN$   M5        44298   200    93    70   128   175   249    618
       M15       19939   393   182   133   251   350   492   1046
       H1         5082   788   361   245   505   703   999   1981
       H3         1913  1265   627   235   760  1151  1674   3338
       D1          520  2560   861  1175  1896  2438  3076   5646
       W1          100  5729  1941  2961  4106  5451  7095  12323

In [21]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm Neg']].describe().round(0).astype(int)

HL Norm Neg                                           
                 count  mean   std    min   25%   50%   75%   max
Symbol TF                                                        
WIN$   M5        42475  -203    95   -654  -255  -180  -130   -70
       M15       19552  -400   185  -1046  -506  -355  -255  -113
       H1         4972  -795   363  -1994 -1008  -714  -512  -245
       H3         1848 -1303   654  -3227 -1698 -1175  -775  -235
       D1          501 -2648   939  -5223 -3224 -2438 -1919 -1163
       W1          115 -5632  1810 -10320 -7036 -5147 -4242 -2831

# Chart

In [22]:
# for SYMBOL, Asset in Src.groupby('Symbol', sort=0):

#     plt.title(f'{SYMBOL} - Candle Size Distribuition', fontweight='bold')

#     for TF, Data in Asset.groupby('TF', sort=0): 
#         # sns.kdeplot(Data['Change'], label=TF)
#         # sns.histplot(Data['Change'], label=TF, stat='count', bins=100, element='step', fill=False)
#         plt.hist(Data['Change'], bins=100, label=TF, histtype='step')


#     plt.ylabel('Frequency')
#     plt.xlabel('Candle Size')

#     plt.yscale('log', base=10)
#     plt.gca().yaxis.set_major_formatter(mpl.ticker.ScalarFormatter())

#     plt.legend()
#     plt.tight_layout()
#     plt.show()

# Snippets

In [23]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['Direct']       = (Src['Close'] - Src['Open']).apply(lambda x: -1 if (x <  0) else  +1)
    Src['Bull Sign']    =                Src['Direct'].apply(lambda x:  x if (x >= 0) else nan)
    Src['Bear Sign']    =                Src['Direct'].apply(lambda x:  x if (x <  0) else nan)

    Src['Change']       = Src['Close'] - Src['Open']
    Src['HL']           = Src['High']  - Src['Low']
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Psc']         = Calc.groupby(['Symbol','TF'], sort=0)['Change'] .expanding(20).rank(ascending=1, pct=1).mul(100)
    Calc['HL Psc']          = Calc.groupby(['Symbol','TF'], sort=0)['HL']     .expanding(20).rank(ascending=1, pct=1).mul(100)
    
    Calc['Chg Norm']        = Calc.apply(lambda x: x['Change'] if (5 < x['Chg Psc'] < 95) else nan, axis=1)
    Calc['HL Norm']         = Calc.apply(lambda x: x['HL']     if (5 < x['HL Psc']  < 95) else nan, axis=1)

    Calc['Chg Norm Abs']    = Calc['Chg Norm'].abs()
    Calc['Chg Norm Pos']    = Calc['Chg Norm'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Chg Norm Neg']    = Calc['Chg Norm'].apply(lambda x: x if (x <  0) else nan)
    
    Calc['HL Norm Pos']     = Calc['HL Norm'] * Calc['Bull Sign']
    Calc['HL Norm Neg']     = Calc['HL Norm'] * Calc['Bear Sign']
    return Calc


Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm Pos']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm Neg']].describe().round(0).astype(int)

Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Abs']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Pos']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Neg']].describe().round(0).astype(int)


Chg Norm Neg                                       
                  count  mean   std   min   25%   50%  75% max
Symbol TF                                                     
WIN$   M5         45128   -80    62  -366  -115   -64  -31  -5
       M15        21167  -153   122  -556  -222  -120  -56  -5
       H1          5482  -315   257 -1237  -458  -247 -111  -5
       H3          2023  -528   456 -2114  -789  -387 -155  -5
       D1           558 -1198   850 -4056 -1652 -1046 -508  -6
       W1           119 -2216  1667 -6542 -3325 -1913 -826 -63